<a href="https://colab.research.google.com/github/MariaFernandaOrtega/Topic-Modelling-Tutorial/blob/main/Bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install bertopic
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.8 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039166 sha256=b6287b13a71a5047f072d7a081d32f72f36cc9878a86f8db8defcba020ab79c6
  Stored in di

In [2]:
# Try to import BERTopic
from bertopic import BERTopic

In [3]:
# Install older version of joblib
!pip install --upgrade joblib==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.3.2
    Uninstalling joblib-1.3.2:
      Successfully uninstalled joblib-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.10.1 requires joblib>=1.1.1, but you have joblib 1.1.0 which is incompatible.
scikit-learn 1.2.2 requires joblib>=1.1.1, but you have joblib 1.1.0 which is incompatible.


In [4]:
# Data processing
import pandas as pd
import numpy as np
import ast

# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

# Topic model
from bertopic import BERTopic

# Dimension reduction
from umap import UMAP

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [384]:
df = pd.read_csv("processed_congress_data.csv")

# keep only the text column

df = df[['processed_text_v2', 'congress', 'period']]



In [29]:
df

,processed_text_v2,congress,period
0,opening statement hon robert menendez u new je...,senate,2023-2024
1,opening statement hon christopher murphy u con...,senate,2023-2024
2,opening statement hon robert menendez u new je...,senate,2023-2024
3,opening statement hon robert menendez u new je...,senate,2023-2024
4,opening statement hon robert menendez u new je...,senate,2023-2024
...,...,...,...
151,opening statement opening statement record sir...,house,2021-2021
152,committee met pursuant notice 208 pm room 2172...,house,2021-2021
153,committee met pursuant notice 1027 room hvc210...,house,2021-2021
154,committee met pursuant notice 538 pm room 2118...,house,2021-2021


In [385]:
# Remove specified words from the 'processed_text_v2' column
words_to_remove = {'well','people','effort','human','year','united','partner','including','one','department','back','like','mr','representative', 'think','want', 'right','thank', 'would', 'know', 'going','senator', 'answer','confirmed', "chairman", "also", "question", "country", "state",'secretary','kerry','time', 'government', 'answer', 'work','member'}

# Define a function to process the text within the lists
def process_text_list(text_list_str, words_to_remove):
    # Convert the string representation of a list into an actual list
    word_list = ast.literal_eval(text_list_str)
    # Remove the specified words and join the remaining words into a single string
    return ' '.join([word for word in word_list if word not in words_to_remove])

# Apply this function to the 'processed_text_v2' column
df['processed_text_v2'] = df['processed_text_v2'].apply(lambda x: process_text_list(x, words_to_remove))

# Display the first few rows after modification
df['processed_text_v2'].head()

0    opening statement hon robert menendez u new je...
1    opening statement hon christopher murphy u con...
2    opening statement hon robert menendez u new je...
3    opening statement hon robert menendez u new je...
4    opening statement hon robert menendez u new je...
Name: processed_text_v2, dtype: object

In [386]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re

# Ensure you have the necessary NLTK data
nltk.download('punkt')



# Define a function to lemmatize specific words and remove numbers
def lemmatize_and_remove_numbers(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Lemmatize specific words and remove numbers
    lemmatized_tokens = []
    for token in tokens:
        if token.lower() in ['europe', 'european']:
            lemmatized_tokens.append('Europe')
        elif token.lower() in ['russia', 'russian']:
            lemmatized_tokens.append('Russia')
        elif not re.fullmatch(r'\d+', token):  # Regex to remove numbers
            lemmatized_tokens.append(token)

    # Reconstruct the text
    return ' '.join(lemmatized_tokens)

# Apply the function to the 'processed_text_v2' column
df['processed_text_v2'] = df['processed_text_v2'].apply(lemmatize_and_remove_numbers)

# Save the modified dataframe
df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,processed_text_v2,congress,period
0,opening statement hon robert menendez u new je...,senate,2023-2024
1,opening statement hon christopher murphy u con...,senate,2023-2024
2,opening statement hon robert menendez u new je...,senate,2023-2024
3,opening statement hon robert menendez u new je...,senate,2023-2024
4,opening statement hon robert menendez u new je...,senate,2023-2024
...,...,...,...
151,opening statement opening statement record sir...,house,2021-2021
152,committee met pursuant notice pm room rayburn ...,house,2021-2021
153,committee met pursuant notice room hvc210 capi...,house,2021-2021
154,committee met pursuant notice pm room rayburn ...,house,2021-2021


Senate all

In [312]:
# Create a subset of the data where 'congress' column is 'senate' and 'period' is '2022-2023'
senate_subset = df[(df['congress'] == 'senate')] #& (df['period'] == '2023-2024')]

senate_subset

,processed_text_v2,congress,period
0,opening statement hon robert menendez u new je...,senate,2023-2024
1,opening statement hon christopher murphy u con...,senate,2023-2024
2,opening statement hon robert menendez u new je...,senate,2023-2024
3,opening statement hon robert menendez u new je...,senate,2023-2024
4,opening statement hon robert menendez u new je...,senate,2023-2024
...,...,...,...
92,committee met pursuant notice room sd106 dirks...,senate,2021-2022
93,briefing held deep background wfb article abra...,senate,2021-2022
94,opening statement hon robert menendez u new je...,senate,2021-2022
95,opening statement hon robert menendez u new je...,senate,2021-2022


In [339]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=5,
                  n_components=3,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model.fit_transform(senate_subset['processed_text_v2'])

In [340]:
# Get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,13,-1_taiwan_china_democracy_russia,"[taiwan, china, democracy, russia, world, supp...",[opening statement hon robert menendez u new j...
1,0,58,0_support_security_assistance_need,"[support, security, assistance, need, internat...",[committee met pursuant notice room sd106 dirk...
2,1,12,1_russia_ukraine_europe_sanction,"[russia, ukraine, europe, sanction, putin, nat...",[opening statement hon robert menendez u new j...


senate 2023-2024

In [341]:
# Create a subset of the data where 'congress' column is 'senate' and 'period' is '2022-2023'
senate_23 = df[(df['congress'] == 'senate') & (df['period'] == '2023-2024')]

senate_23

,processed_text_v2,congress,period
0,opening statement hon robert menendez u new je...,senate,2023-2024
1,opening statement hon christopher murphy u con...,senate,2023-2024
2,opening statement hon robert menendez u new je...,senate,2023-2024
3,opening statement hon robert menendez u new je...,senate,2023-2024
4,opening statement hon robert menendez u new je...,senate,2023-2024
5,opening statement hon robert menendez u new je...,senate,2023-2024
6,opening statement hon benjamin l cardin u mary...,senate,2023-2024
7,opening statement hon christopher murphy u con...,senate,2023-2024
8,opening statement hon robert menendez u new je...,senate,2023-2024
9,opening statement hon jeanne shaheen u new ham...,senate,2023-2024


In [344]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=5,
                  n_components=3,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model_23 = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model_23.fit_transform(senate_23['processed_text_v2'])

In [345]:
# Get the list of topics
topic_model_23.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16,-1_russia_usaid_support_ukraine,"[russia, usaid, support, ukraine, china, secur...",[opening statement hon robert menendez u new j...


Senate 2022-2021

In [346]:
# Create a subset of the data where 'congress' column is 'senate' and 'period' is '2022-2023'
senate_21 = df[(df['congress'] == 'senate') & (df['period'] == '2021-2022')]

senate_21

,processed_text_v2,congress,period
30,opening statement hon james e risch u idaho se...,senate,2021-2022
31,committee met pursuant notice pm russell senat...,senate,2021-2022
32,opening statement hon bob menendez u new jerse...,senate,2021-2022
33,opening statement hon jeanne shaheen u new ham...,senate,2021-2022
34,opening statement hon bob menendez u new jerse...,senate,2021-2022
...,...,...,...
92,committee met pursuant notice room sd106 dirks...,senate,2021-2022
93,briefing held deep background wfb article abra...,senate,2021-2022
94,opening statement hon robert menendez u new je...,senate,2021-2022
95,opening statement hon robert menendez u new je...,senate,2021-2022


In [373]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=7,
                  n_components=3,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model_21= BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model_21.fit_transform(senate_21['processed_text_v2'])

In [374]:
# Get the list of topics
topic_model_21.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,27,-1_security_support_afghanistan_international,"[security, support, afghanistan, international...",[opening statement hon robert menendez u new j...
1,0,26,0_security_support_usaid_need,"[security, support, usaid, need, foreign, assi...",[briefing held deep background wfb article abr...
2,1,14,1_russia_ukraine_sanction_europe,"[russia, ukraine, sanction, europe, colombia, ...",[opening statement hon robert menendez u new j...


House all

In [387]:
# Create a subset of the data where 'congress' column is 'senate' and 'period' is '2022-2023'
house_subset = df[(df['congress'] == 'house')] #& (df['period'] == '2023-2024')]

house_subset

,processed_text_v2,congress,period
16,committee met pursuant notice room house visit...,house,2023-2024
17,committee met pursuant notice pm room house vi...,house,2023-2024
18,committee met pursuant notice room house visit...,house,2023-2024
19,subcommittee met pursuant notice pm room raybu...,house,2023-2024
20,subcommittee met pursuant notice pm room house...,house,2023-2024
...,...,...,...
151,opening statement opening statement record sir...,house,2021-2021
152,committee met pursuant notice pm room rayburn ...,house,2021-2021
153,committee met pursuant notice room hvc210 capi...,house,2021-2021
154,committee met pursuant notice pm room rayburn ...,house,2021-2021


In [404]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=3,
                  n_components=9,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model_2 = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model_2.fit_transform(house_subset['processed_text_v2'])

In [405]:
# Get the list of topics
topic_model_2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7,-1_energy_climate_europe_wind,"[energy, climate, europe, wind, arctic, keatin...",[subcommittee met pursuant notice via webex ho...
1,0,37,0_russia_support_war_need,"[russia, support, war, need, ukraine, congress...",[committee met pursuant notice pm room rayburn...
2,1,17,1_china_nato_europe_region,"[china, nato, europe, region, need, important,...",[committee met pursuant notice room house visi...
3,2,12,2_africa_syria_much_need,"[africa, syria, much, need, burma, support, re...",[subcommittee met pursuant notice via webex ho...


house 2023-2024

In [406]:
# Create a subset of the data where 'congress' column is 'senate' and 'period' is '2022-2023'
house_23 = df[(df['congress'] == 'house') & (df['period'] == '2023-2024')]

house_23

,processed_text_v2,congress,period
16,committee met pursuant notice room house visit...,house,2023-2024
17,committee met pursuant notice pm room house vi...,house,2023-2024
18,committee met pursuant notice room house visit...,house,2023-2024
19,subcommittee met pursuant notice pm room raybu...,house,2023-2024
20,subcommittee met pursuant notice pm room house...,house,2023-2024
21,committee met pursuant notice room house visit...,house,2023-2024
22,committee met pursuant notice room house visit...,house,2023-2024
23,committee met pursuant notice room house visit...,house,2023-2024
24,committee met pursuant notice room house visit...,house,2023-2024
25,committee met pursuant notice room house visit...,house,2023-2024


In [426]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=7,
                  n_components=3,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model_h23= BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model_h23.fit_transform(house_23['processed_text_v2'])

In [427]:
# Get the list of topics
topic_model_h23.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,14,-1_ukraine_support_russia_china,"[ukraine, support, russia, china, need, war, m...",[subcommittee met pursuant notice pm room hous...


House 2021-2022

In [428]:
# Create a subset of the data where 'congress' column is 'senate' and 'period' is '2022-2023'
house_21 = df[(df['congress'] == 'house') & (df['period'] == '2021-2021')]

house_21

,processed_text_v2,congress,period
97,subcommittee met pursuant notice room rayburn ...,house,2021-2021
98,subcommittee met pursuant notice via webex hon...,house,2021-2021
99,committee met pursuant notice via webex hon gr...,house,2021-2021
100,committee met pursuant notice room rayburn hou...,house,2021-2021
101,committee met pursuant notice via webex hon th...,house,2021-2021
102,subcommittee met pursuant notice via webex hon...,house,2021-2021
103,subcommittee met pursuant notice via webex hon...,house,2021-2021
104,committee met pursuant notice pm via webex hon...,house,2021-2021
105,committee met pursuant notice pm via webex hon...,house,2021-2021
106,subcommittee met pursuant notice room rayburn ...,house,2021-2021


In [444]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=3,
                  n_components=2,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model_h21= BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model_h21.fit_transform(house_21['processed_text_v2'])

In [445]:
# Get the list of topics
topic_model_h21.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12,-1_russia_energy_europe_climate,"[russia, energy, europe, climate, keating, ukr...",[subcommittee met pursuant notice via webex ho...
1,0,32,0_need_president_support_much,"[need, president, support, much, congress, get...",[subcommittee met pursuant notice via webex ho...
2,1,15,1_china_nato_region_europe,"[china, nato, region, europe, need, much, issu...",[subcommittee met pursuant notice via webex ho...
